In [13]:
# Importar las bibliotecas necesarias
from ultralytics import YOLO
import pydicom
import numpy as np
from PIL import Image, ImageDraw

# Ruta al modelo entrenado
model_path = '/Users/macdenico/PycharmProjects/PMM-FALP2024/Model Training and Validation/deteccion/modelos/best.pt'

# Ruta a la imagen DICOM
image_path = '/Users/macdenico/PycharmProjects/PMM-FALP2024/Model Training and Validation/deteccion/images/2fc787f72bfa6b6c0bbc0a85413c8a9f.dicom'

# Cargar el modelo YOLO
model = YOLO(model_path)

# Leer la imagen DICOM
dicom = pydicom.dcmread(image_path)

# Obtener el arreglo de píxeles de la imagen DICOM
pixel_array = dicom.pixel_array

# Normalizar el arreglo de píxeles a valores entre 0 y 255
pixel_array = pixel_array.astype(float)
scaled_pixel_array = (np.maximum(pixel_array - np.min(pixel_array), 0) / (np.max(pixel_array) - np.min(pixel_array))) * 255.0
scaled_pixel_array = np.uint8(scaled_pixel_array)

# Manejar la interpretación fotométrica MONOCHROME1 si es necesario
if dicom.PhotometricInterpretation == "MONOCHROME1":
    scaled_pixel_array = np.max(scaled_pixel_array) - scaled_pixel_array

# Convertir el arreglo de píxeles en una imagen PIL
image = Image.fromarray(scaled_pixel_array)

# Si la imagen es en escala de grises, convertirla a RGB
if image.mode != 'RGB':
    image = image.convert('RGB')

# Realizar la inferencia utilizando el modelo YOLO
result = model.predict(image, conf=0.2, verbose=False, imgsz=1280)[0]

# Mostrar los resultados de la inferencia
print(result)

# Obtener las cajas delimitadoras de las detecciones
boxes = result.boxes

# Crear una máscara vacía con las mismas dimensiones que la imagen original
mask = Image.new('L', image.size, 0)  # 'L' para modo de imagen en escala de grises

# Dibujar las cajas delimitadoras en la máscara
if boxes is not None and len(boxes) > 0:
    # Convertir las cajas a un array de NumPy
    boxes_array = boxes.xyxy.cpu().numpy()  # Convertir a CPU y luego a NumPy

    draw = ImageDraw.Draw(mask)

    for box in boxes_array:
        # Obtener las coordenadas de la caja y asegurarse de que son enteros
        x1, y1, x2, y2 = map(int, box[:4])
        # Dibujar un rectángulo lleno (valor 255) en la máscara
        draw.rectangle([x1, y1, x2, y2], outline=255, fill=255)

    # Superponer la máscara coloreada sobre la imagen original
    # Convertir la imagen original a modo RGBA
    image_rgba = image.convert('RGBA')

    # Crear una imagen del color de la máscara (por ejemplo, rojo semitransparente)
    mask_color = (255, 0, 0, 100)  # (R, G, B, Alpha)

    # Crear una imagen del color de la máscara con transparencia
    colored_mask = Image.new('RGBA', image.size, mask_color)

    # Usar la máscara como canal alfa
    mask_np = np.array(mask)
    alpha_mask = (mask_np / 255) * mask_color[3]  # Escalar el valor de alfa según la máscara

    # Reemplazar el canal alfa en la máscara coloreada
    colored_mask.putalpha(Image.fromarray(alpha_mask.astype('uint8')))

    # Superponer la máscara coloreada sobre la imagen original
    combined = Image.alpha_composite(image_rgba, colored_mask)

    # Mostrar la imagen combinada
    combined.show()

    # Guardar la imagen combinada si lo deseas
    #combined.save('/ruta/donde/guardar/imagen_con_mascara.png')

else:
    print("No se encontraron detecciones en la imagen.")


ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'mass'}
obb: None
orig_img: array([[[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [62, 62, 62],
        [63, 63, 63],
        [64, 64, 64]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [61, 61, 61],
        [62, 62, 62],
        [62, 62, 62]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [62, 62, 62],
        [63, 63, 63],
        [64, 64, 64]],

       ...,

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,

FileNotFoundError: [Errno 2] No such file or directory: '/ruta/donde/guardar/imagen_con_mascara.png'